In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv("try.env"))

True

In [2]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [3]:
import os
api_key = os.environ['GOOGLE_API_KEY']

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [5]:
from langchain import PromptTemplate

In [6]:
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [27]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [26]:
URL = "https://blog.google/technology/ai/google-gemini-ai/#sundar-note"

In [9]:
loader = WebBaseLoader(URL)
docs = loader.load()

In [10]:
# Extract the text from the website data document
text_content = docs[0].page_content

# The text content between the substrings "code, audio, image and video." to
# "Cloud TPU v5p" is relevant for this tutorial. You can use Python's `split()`
# to select the required content.
if(len(text_content.split("code, audio, image and video.",1)) > 1):
    text_content_1 = text_content.split("code, audio, image and video.",1)[1]
    final_text = text_content_1.split("Cloud TPU v5p",1)[0]

else:
    final_text = text_content.split("Cloud TPU v5p",1)[0]

# Convert the text to LangChain's `Document` format
docs =  [Document(page_content=final_text, metadata={"source": "local"})]

In [11]:
vectorstore = Chroma.from_documents(
                     documents=docs,                 # Data
                     embedding=embeddings,    # Embedding model
                     persist_directory="./chroma_db" # Directory to save data
                     )

In [12]:
# Load from disk
vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",       # Directory of db
                        embedding_function=embeddings   # Embedding model
                   )
retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 1})
print(len(retriever.get_relevant_documents("MMLU")))

C:\Users\tript\AppData\Local\Temp\ipykernel_10892\883099469.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore_disk = Chroma(
C:\Users\tript\AppData\Local\Temp\ipykernel_10892\883099469.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(len(retriever.get_relevant_documents("MMLU")))


1


In [ ]:
llm = ChatGoogleGenerativeAI(model = "gemini-pro",temperature = 0.75, top_p = 0.85)

In [14]:
llm_prompt_template = """You are an assistant for question-answering tasks.
Use the following context to answer the question.
If you don't know the answer, then refer to your own intelligence.
Use five sentences maximum and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} template="You are an assistant for question-answering tasks.\nUse the following context to answer the question.\nIf you don't know the answer, then refer to your own intelligence.\nUse five sentences maximum and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:"


In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [17]:
Question = input("What question do you have in your mind on reading it?")

In [18]:
rag_chain.invoke(Question)

'Pegasus is a mythical winged horse in Greek mythology, often depicted as pure white in color. It was sired by Poseidon, the god of the sea, and foaled by the Gorgon Medusa.'

In [19]:
doc_prompt = PromptTemplate.from_template("{page_content}")

# To query Gemini
template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(template)

print(prompt)

input_variables=['text'] input_types={} partial_variables={} template='Write a concise summary of the following:\n"{text}"\nCONCISE SUMMARY:'


In [20]:
stuff_chain = (
    # Extract data from the documents and add to the key `text`.
    {
        "text": lambda docs: "\n\n".join(
            format_document(doc, doc_prompt) for doc in docs
        )
    }
    | prompt         # Prompt for Gemini
    | llm                # Gemini function
    | StrOutputParser()  # output parser
)

In [21]:
stuff_chain.invoke(docs)

"Google's Gemini, a large-scale AI model, surpasses state-of-the-art performance on various benchmarks, including language understanding, reasoning, coding, and competitive programming. Gemini's native multimodality enables it to seamlessly understand and reason across different inputs, including text, images, and audio. It excels in sophisticated reasoning, extracting insights from complex data, and explaining reasoning in subjects like math and physics. Gemini's scalability, efficiency, and reliability make it suitable for various applications in fields such as science, finance, and software development."

In [22]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType



In [23]:
def langchain_agent():
    llm = ChatGoogleGenerativeAI(model = "gemini-pro",temperature = 0.75, top_p = 0.85)
    tools = load_tools([ "wikipedia", "llm-math"], llm = llm)
    agent = initialize_agent(
        tools,llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True
    )
  
    
    result = agent.run("What is GenAI?")
    
    print(result)
   

In [24]:
if __name__ == "__main__":
    langchain_agent()

C:\Users\tript\AppData\Local\Temp\ipykernel_10892\1314820065.py:4: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(
C:\Users\tript\AppData\Local\Temp\ipykernel_10892\1314820065.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run("What is GenAI?")




> Entering new AgentExecutor chain...
I do not know what GenAI is, but I can search the web for it.
Action: wikipedia
Action Input: What is GenAI?
Observation: Page: Generative artificial intelligence
Summary: Generative artificial intelligence (generative AI, GenAI, or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models often generate output in response to specific prompts. Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data. 
Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora. Companies such